In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from pathlib import Path

import tensorflow as tf
from keras._tf_keras import keras

2024-12-04 19:51:35.506929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733313095.587070   51609 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733313095.610071   51609 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
RANDOM_SEED = 709

BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data"
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"
CATEGORIES = ["chihuahua", "muffin"]
img_w, img_h, img_ch = IMAGE_TARGET_SIZE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 3

## Load data

In [4]:
train_ds: tf.data.Dataset = keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=RANDOM_SEED,
    image_size=(img_h, img_w),
    batch_size=BATCH_SIZE,
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

Found 4733 files belonging to 2 classes.
Using 3787 files for training.


I0000 00:00:1733313105.922546   51609 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5558 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [5]:
val_ds: tf.data.Dataset = keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(img_h, img_w),
    batch_size=BATCH_SIZE,
)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Found 4733 files belonging to 2 classes.
Using 946 files for validation.


In [6]:
total_train = len(train_ds) * BATCH_SIZE
total_val = len(val_ds) * BATCH_SIZE

## Model definition

In [7]:
model = keras.Sequential(
    [
        keras.layers.Input(IMAGE_TARGET_SIZE),
        keras.layers.Rescaling(1 / 255),
        keras.layers.Conv2D(32, 3, activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(1),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1577088)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   201,867,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,868,417 (770.07 MB)

 Trainable params: 201,868,417 (770.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tensorboard = keras.callbacks.TensorBoard(
    str(BASE_DIR / ".tensorboard"),
    histogram_freq=5,
    write_images=True,
)

learning_rate = 1e-3
sgd = keras.optimizers.SGD(learning_rate)
loss = keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(
    loss=loss,
    optimizer=sgd,
    metrics=["accuracy"],
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    steps_per_epoch=total_train // BATCH_SIZE,
    validation_steps=total_val // BATCH_SIZE,
    callbacks=[tensorboard],
)
results = history.history
results

Epoch 1/3


I0000 00:00:1733313121.509219   51909 service.cc:148] XLA service 0x7f0fa4006360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733313121.509565   51909 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
I0000 00:00:1733313121.733323   51909 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/119 ━━━━━━━━━━━━━━━━━━━━ 6:13 3s/step - accuracy: 0.5625 - loss: 0.6865

I0000 00:00:1733313124.006220   51909 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5563 - loss: 0.6978